### Decorators Application (Memoization)

Вернемся к нашему примеру с числами Фибоначчи:

In [1]:
def fib(n):
    print ('Calculating fib({0})'.format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

Запустив этот алгоритм, мы увидим, что он совершенно неэффективен, поскольку одни и те же числа Фибоначчи вычисляются несколько раз:

In [2]:
fib(6)

Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)


8

Было бы лучше, если бы мы могли как-то «сохранить» эти результаты, так что если мы уже вычислили `fib(4)` и `fib(3)`, мы могли бы просто вызвать эти значения при вычислении `fib(5) = fib(4) + fib(3)` вместо того, чтобы пересчитывать их.

Эта концепция повышения эффективности нашего кода путем кэширования предварительно вычисленных значений, чтобы их не нужно было пересчитывать каждый раз, называется «мемориализацией»

Мы можем решить эту задачу, используя простой класс и словарь, в котором хранятся все уже вычисленные числа Фибоначчи:

In [3]:
class Fib:
    def __init__(self):
        self.cache = {1: 1, 2: 1}

    def fib(self, n):
        if n not in self.cache:
            print('Calculating fib({0})'.format(n))
            self.cache[n] = self.fib(n-1) + self.fib(n-2)
        return self.cache[n]

In [4]:
f = Fib()

In [5]:
f.fib(1)

1

In [6]:
f.fib(6)

Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


8

In [7]:
f.fib(7)

Calculating fib(7)


13

Давайте посмотрим, как это можно переписать с использованием замыкания:

In [8]:
def fib():
    cache = {1: 1, 2: 2}

    def calc_fib(n):
        if n not in cache:
            print('Calculating fib({0})'.format(n))
            cache[n] = calc_fib(n-1) + calc_fib(n-2)
        return cache[n]

    return calc_fib

In [9]:
f = fib()

In [10]:
f(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


89

Теперь давайте посмотрим, как это реализовать с помощью декоратора:

In [11]:
from functools import wraps

def memoize_fib(fn):
    cache = dict()

    @wraps(fn)
    def inner(n):
        if n not in cache:
            cache[n] = fn(n)
        return cache[n]

    return inner

In [12]:
@memoize_fib
def fib(n):
    print ('Calculating fib({0})'.format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [13]:
fib(3)

Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


2

In [14]:
fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)


55

In [15]:
fib(6)

8

Как вы можете видеть, мы обращаемся к кэшу, когда значения доступны.

Теперь мы сделали наш декоратор мемоизации «жестко закодированным» для функций с одним аргументом — мы могли бы сделать его более универсальным.

Например, чтобы обработать произвольное количество позиционных аргументов и аргументов, состоящих только из ключевых слов, мы могли бы сделать следующее:

In [44]:
def memoize(fn):
    cache = dict()

    @wraps(fn)
    def inner(*args):
        if args not in cache:
            cache[args] = fn(*args)
        return cache[args]

    return inner

In [17]:
@memoize
def fib(n):
    print ('Calculating fib({0})'.format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [18]:
fib(6)

Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


8

In [19]:
fib(7)

Calculating fib(7)


13

Конечно, с помощью этого довольно универсального декоратора мемоизации мы можем запоминать и другие функции:

In [20]:
def fact(n):
    print('Calculating {0}!'.format(n))
    return 1 if n < 2 else n * fact(n-1)

In [21]:
fact(5)

Calculating 5!
Calculating 4!
Calculating 3!
Calculating 2!
Calculating 1!


120

In [22]:
fact(5)

Calculating 5!
Calculating 4!
Calculating 3!
Calculating 2!
Calculating 1!


120

И запоминание:

In [23]:
@memoize
def fact(n):
    print('Calculating {0}!'.format(n))
    return 1 if n < 2 else n * fact(n-1)

In [24]:
fact(6)

Calculating 6!
Calculating 5!
Calculating 4!
Calculating 3!
Calculating 2!
Calculating 1!


720

In [25]:
fact(6)

720

Однако у нашего простого мемоайзера есть недостаток:
* размер кэша не ограничен — вероятно, это нехорошо! В общем, мы хотим ограничить кэш определенным количеством записей, сбалансировав вычислительную эффективность с использованием памяти.
* мы не обрабатываем **kwargs

Мемоизация — настолько распространенное явление, что в Python даже есть декоратор мемоизации, созданный для нас!

Он находится в, как вы уже догадались, модуле **functools** и называется **lru_cache**, и будет намного эффективнее по сравнению с простым примером мемоизации, который мы сделали выше.

[LRU-кэш = кэширование наименее недавно использованных записей: поскольку кэш не безграничен, в какой-то момент кэшированные записи необходимо отбросить, и наименее недавно использованные записи отбрасываются первыми]

In [26]:
from functools import lru_cache

In [27]:
@lru_cache()
def fact(n):
    print("Calculating fact({0})".format(n))
    return 1 if n < 2 else n * fact(n-1)

In [28]:
fact(5)

Calculating fact(5)
Calculating fact(4)
Calculating fact(3)
Calculating fact(2)
Calculating fact(1)


120

In [29]:
fact(4)

24

Как видите, `fact(4)` был возвращён через кэшированную запись!

То же самое и с нашей функцией Фибоначчи:

In [30]:
@lru_cache()
def fib(n):
    print("Calculating fib({0})".format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [31]:
fib(6)

Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


8

In [32]:
fib(5)

5

Вспомните несколько видео назад, где мы засекали время расчета чисел Фибоначчи. Расчет fib(35) занимал несколько секунд - каждый раз...

In [33]:
from time import perf_counter

In [34]:
def fib_no_memo(n):
    return 1 if n < 3 else fib_no_memo(n-1) + fib_no_memo(n-2)

In [35]:
start = perf_counter()
result = fib_no_memo(35)
print("result={0}, elapsed: {1}s".format(result, perf_counter() - start))

result=9227465, elapsed: 2.939012289158911s


In [36]:
@lru_cache()
def fib_memo(n):
    return 1 if n < 3 else fib_memo(n-1) + fib_memo(n-2)

In [37]:
start = perf_counter()
result = fib_memo(35)
print("result={0}, elapsed: {1}s".format(result, perf_counter() - start))

result=9227465, elapsed: 9.83349429017899e-05s


И если мы сделаем вызовы снова:

In [38]:
start = perf_counter()
result = fib_no_memo(35)
print("result={0}, elapsed: {1}s".format(result, perf_counter() - start))

result=9227465, elapsed: 2.782454120518548s


In [39]:
start = perf_counter()
result = fib_memo(35)
print("result={0}, elapsed: {1}s".format(result, perf_counter() - start))

result=9227465, elapsed: 5.6617088337596044e-05s


Вы могли заметить, что декоратор `lru_cache` был реализован с использованием `()` — мы рассмотрим это подробнее позже, но это потому, что декораторы сами по себе могут иметь параметры (помимо декорируемой функции).

Одним из аргументов декоратора `lru_cache` является размер кэша — по умолчанию он равен 128 элементам, но мы можем легко это изменить — из соображений производительности используйте для размера кэша степени числа 2 (или None для неограниченного кэша):

In [40]:
@lru_cache(maxsize=8)
def fib(n):
    print("Calculating fib({0})".format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [41]:
fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


55

In [42]:
fib(20)

Calculating fib(20)
Calculating fib(19)
Calculating fib(18)
Calculating fib(17)
Calculating fib(16)
Calculating fib(15)
Calculating fib(14)
Calculating fib(13)
Calculating fib(12)
Calculating fib(11)


6765

In [43]:
fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


55

Обратите внимание, как Python пришлось пересчитать `fib` для `10, 9,` и т. д. Это связано с тем, что кэш может содержать только 10 элементов, поэтому, когда мы вычислили `fib(20)`, он сохранил fib для `20, 19, ..., 11` (10 элементов), и поэтому самые старые элементы fib `10, 9, ..., 1` были удалены из кэша, чтобы освободить место.

---